In [4]:
import torch
from micrograd.engine import Value
from graphviz import Digraph

Function for drawing nodes

In [5]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

Testing of Engine for Values

In [8]:
# Using Value 

# Forward Pass
x = Value(-4.0)
z = 2 * x + 2 + x
q = z.relu() + z * x
h = (z * z).relu()
y = h + q + q * x

print(f"""" Forward Pass
      x = {x},
      z = {z},
      q = {q},
      h = {h},
      y = {y}
      """)

#Backward pass
y.backward()
xmg, ymg = x, y

print(f"""" Backward Pass
      xmg = {x},
      ymg = {y}
      """)

" Forward Pass
      x = Value(data=-4.0, grad=0),
      z = Value(data=-10.0, grad=0),
      q = Value(data=40.0, grad=0),
      h = Value(data=100.0, grad=0),
      y = Value(data=-20.0, grad=0)
      
" Backward Pass
      xmg = Value(data=-4.0, grad=46.0),
      ymg = Value(data=-20.0, grad=1)
      


In [12]:
# Using PyTorch
# Forward Pass
x = torch.Tensor([-4.0]).double()
x.requires_grad = True
z = 2 * x + 2 + x
q = z.relu() + z * x
h = (z * z).relu()
y = h + q + q * x

print(f"""" Forward Pass
      x = {x},
      z = {z},
      q = {q},
      h = {h},
      y = {y}
      """)

y.backward()
xpt, ypt = x, y

print(f"""" Backward Pass
      xpt =  {x.grad}
      ypt =  {y.grad}
      """)


" Forward Pass
      x = tensor([-4.], dtype=torch.float64, requires_grad=True),
      z = tensor([-10.], dtype=torch.float64, grad_fn=<AddBackward0>),
      q = tensor([40.], dtype=torch.float64, grad_fn=<AddBackward0>),
      h = tensor([100.], dtype=torch.float64, grad_fn=<ReluBackward0>),
      y = tensor([-20.], dtype=torch.float64, grad_fn=<AddBackward0>)
      
" Backward Pass
      xpt = tensor([-4.], dtype=torch.float64, requires_grad=True), tensor([46.], dtype=torch.float64)
      ypt = tensor([-20.], dtype=torch.float64, grad_fn=<AddBackward0>), None
      


C:\Users\hunai\AppData\Local\Temp\ipykernel_21128\909692138.py:23: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  ypt = {y}, {y.grad}
